In [2]:
import biblio_modules

import datetime
from dateutil import parser
import time
import pytz

import pyarrow as py
import pandas as pd
import numpy as np

In [3]:
transactions_processing = pd.read_csv("../Source/Transactions.csv", sep=";")
transactions_processing

,id_prod,date,session_id,client_id
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329
1,0_1390,2021-03-01 00:02:26.047414,s_2,c_664
2,0_1352,2021-03-01 00:02:38.311413,s_3,c_580
3,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912
4,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033
...,...,...,...,...
687529,1_508,2023-02-28 23:49:03.148402,s_348444,c_3573
687530,2_37,2023-02-28 23:51:29.318531,s_348445,c_50
687531,1_695,2023-02-28 23:53:18.929676,s_348446,c_488
687532,0_1547,2023-02-28 23:58:00.107815,s_348447,c_4848


<div style="background-color: RGB(100,150,00);" >
<h2 style="margin: auto; padding: 20px; color:#fff; "> Traitement Timestamp  </h2>
<h3 style="margin: auto; padding: 10px; color:#fff; "> --> Début du data processing  </h3>
</div>

In [4]:
#Je coupe la colonne date et time en deux colonnes
transactions_processing[['Date','Time']] = transactions_processing['date'].str.split(expand=True)
#Je converti les date en DateTime
transactions_processing['Date'] = pd.to_datetime(transactions_processing['Date'])

In [5]:
#J'ajoute 1 jour au date sur lesquelles l'heures est de 24
transactions_processing.loc[transactions_processing['Time'].str.startswith('24'), 'Date'] = transactions_processing.loc[transactions_processing['Time'].str.startswith('24'), 'Date'] + pd.DateOffset(days = 1)


In [6]:
#Je change les time '24' en '00' pour uniformiser les times
condition = transactions_processing['Time'].str.startswith('24')
transactions_processing.loc[condition, 'Time'] = '00' + transactions_processing.loc[condition, 'Time'].str[2:]
transactions_processing[transactions_processing['date'].str.contains(' 24:')]


,id_prod,date,session_id,client_id,Date,Time
198325,1_400,2021-10-02 24:04:58.422464,s_107311,c_2513,2021-10-03,00:04:58.422464
198326,1_601,2021-10-02 24:04:59.254020,s_107311,c_2513,2021-10-03,00:04:59.254020
198327,1_735,2021-10-02 24:04:59.422464,s_107311,c_2513,2021-10-03,00:04:59.422464
198328,1_351,2021-10-02 24:04:59.913108,s_107311,c_2513,2021-10-03,00:04:59.913108
198329,1_400,2021-10-02 24:06:02.588166,s_107205,c_5036,2021-10-03,00:06:02.588166
...,...,...,...,...,...,...
221108,1_445,2021-10-26 24:36:01.240212,s_105041,c_3949,2021-10-27,00:36:01.240212
221109,1_296,2021-10-26 24:36:21.240212,s_105041,c_3949,2021-10-27,00:36:21.240212
221820,1_45,2021-10-27 24:18:21.505552,s_98882,c_7656,2021-10-28,00:18:21.505552
221821,1_431,2021-10-27 24:20:46.744883,s_99594,c_8073,2021-10-28,00:20:46.744883


In [7]:
#Je reconvertis mes date en string pour concatener les deux colonnes
transactions_processing['Date'] = transactions_processing['Date'].astype(str)
transactions_processing.head()

,id_prod,date,session_id,client_id,Date,Time
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329,2021-03-01,00:01:07.843138
1,0_1390,2021-03-01 00:02:26.047414,s_2,c_664,2021-03-01,00:02:26.047414
2,0_1352,2021-03-01 00:02:38.311413,s_3,c_580,2021-03-01,00:02:38.311413
3,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912,2021-03-01,00:04:54.559692
4,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033,2021-03-01,00:05:18.801198


In [8]:
transactions_processing[['Year','Month','Day']] = transactions_processing['Date'].str.split(pat="-", expand=True)
transactions_processing[['Hour','Minute','Second']] = transactions_processing['Time'].str.split(pat=":", expand=True)
transactions_processing[['Second','Ns']] = transactions_processing['Second'].str.split(pat=".", expand=True)
transactions_processing.drop(columns=['date', 'Date', 'Time'], inplace=True)
transactions_processing

,id_prod,session_id,client_id,Year,Month,Day,Hour,Minute,Second,Ns
0,0_1259,s_1,c_329,2021,03,01,00,01,07,843138
1,0_1390,s_2,c_664,2021,03,01,00,02,26,047414
2,0_1352,s_3,c_580,2021,03,01,00,02,38,311413
3,0_1458,s_4,c_7912,2021,03,01,00,04,54,559692
4,0_1358,s_5,c_2033,2021,03,01,00,05,18,801198
...,...,...,...,...,...,...,...,...,...,...
687529,1_508,s_348444,c_3573,2023,02,28,23,49,03,148402
687530,2_37,s_348445,c_50,2023,02,28,23,51,29,318531
687531,1_695,s_348446,c_488,2023,02,28,23,53,18,929676
687532,0_1547,s_348447,c_4848,2023,02,28,23,58,00,107815


In [9]:
transactions_processing.columns

Index(['id_prod', 'session_id', 'client_id', 'Year', 'Month', 'Day', 'Hour',
       'Minute', 'Second', 'Ns'],
      dtype='object')

In [10]:
transactions_processing[['Year', 'Month', 'Day', 'Hour','Minute', 'Second', 'Ns']] = transactions_processing[['Year', 'Month', 'Day', 'Hour','Minute', 'Second', 'Ns']].astype(int)

In [11]:
transactions_processing['Timestamp'] = transactions_processing.apply(
    lambda x: pd.Timestamp(year=x.Year,month=x.Month,day=x.Day,hour=x.Hour,minute=x.Minute,second=x.Second,microsecond=x.Ns), axis=1)
transactions_processing.drop(columns=['Year', 'Month', 'Day', 'Hour','Minute', 'Second', 'Ns'], inplace=True)

In [12]:
#transactions_processing.set_index(transactions_processing['Timestamp'], inplace=True)
#transactions_processing.drop(columns='Timestamp', inplace=True)

In [13]:
transactions_processing.head()

,id_prod,session_id,client_id,Timestamp
0,0_1259,s_1,c_329,2021-03-01 00:01:07.843138
1,0_1390,s_2,c_664,2021-03-01 00:02:26.047414
2,0_1352,s_3,c_580,2021-03-01 00:02:38.311413
3,0_1458,s_4,c_7912,2021-03-01 00:04:54.559692
4,0_1358,s_5,c_2033,2021-03-01 00:05:18.801198


In [14]:
transactions_processing.index.dtype

dtype('int64')

In [15]:
transactions_processing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687534 entries, 0 to 687533
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     687534 non-null  object        
 1   session_id  687534 non-null  object        
 2   client_id   687534 non-null  object        
 3   Timestamp   687534 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 21.0+ MB


In [16]:
transactions_processing = transactions_processing[['id_prod', 'Timestamp', 'session_id', 'client_id']]


In [17]:
transactions_processing

,id_prod,Timestamp,session_id,client_id
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329
1,0_1390,2021-03-01 00:02:26.047414,s_2,c_664
2,0_1352,2021-03-01 00:02:38.311413,s_3,c_580
3,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912
4,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033
...,...,...,...,...
687529,1_508,2023-02-28 23:49:03.148402,s_348444,c_3573
687530,2_37,2023-02-28 23:51:29.318531,s_348445,c_50
687531,1_695,2023-02-28 23:53:18.929676,s_348446,c_488
687532,0_1547,2023-02-28 23:58:00.107815,s_348447,c_4848


In [18]:
#je dois donc réculer d'1h tout les enregistrement entre le 28 mars 2021 à 2h du matin et le 31 octobre 2021 à 3h du matin

In [36]:
x= transactions_processing.loc[
    (transactions_processing['Timestamp'] > datetime.datetime(year=2021, month=10, day=31, hour=2 )) 
    & 
    (transactions_processing['Timestamp'] < datetime.datetime(year=2021, month=10, day=31, hour=3))]
x

,id_prod,Timestamp,session_id,client_id
224942,1_202,2021-10-31 02:00:06.704413,s_112515,c_7342
224943,0_1611,2021-10-31 02:01:09.803700,s_112516,c_5756
224944,0_1525,2021-10-31 02:01:47.590195,s_112515,c_7342
224945,1_451,2021-10-31 02:03:02.978253,s_112517,c_8112
224946,0_2074,2021-10-31 02:03:18.728733,s_112515,c_7342
...,...,...,...,...
225011,1_631,2021-10-31 02:53:41.989579,s_112542,c_5041
225012,0_2052,2021-10-31 02:55:21.051240,s_112543,c_3507
225013,1_381,2021-10-31 02:56:36.805619,s_112535,c_161
225014,1_312,2021-10-31 02:57:48.279426,s_112543,c_3507


In [ ]:
transactions_processing.loc[
    (transactions_processing['Timestamp'] > datetime.datetime(year=2021, month=3, day=28, hour=2 )) 
    & 
    (transactions_processing['Timestamp'] < datetime.datetime(year=2021, month=10, day=31, hour=4))]

In [16]:
transactions_processing.to_csv("../Source/Data_processed/timestamps_processed.csv", sep=";", index=False)

<div style="background-color: RGB(100,150,00);" >
<h2 style="margin: auto; padding: 20px; color:#fff; "> Traitement Timestamp  </h2>
<h3 style="margin: auto; padding: 10px; color:#fff; "> --> Fin du data processing  </h3>
</div>

#Je concatene les deux colonnes avec un espace
transactions_processing['Timestamp'] = transactions_processing['Date'] + ' ' + transactions_processing['Time']
transactions_processing
#Je converti ma colonne final en DataTime(ns)
transactions_processing['Timestamp'] = pd.Timestamp(transactions_processing['Timestamp'], errors='raise')
transactions_processing.info()
#J'enlève les colonnes inutile
transactions_processing.drop(columns=['date', 'Date', 'Time'], inplace=True)
#transactions_processing.loc[transactions_processing['Timestamp'].duplicated(keep='last'), 'Timestamp'] = transactions_processing.loc[transactions_processing['Timestamp'].duplicated(keep='last'), 'Timestamp'] + pd.DateOffset(seconds=+1)
transactions_processing['Timestamp'].nunique()
# Remplacer toutes les instances de 2024-06-01T20:40:18.761Z par 2024-06-01T20:40:18.761Z dans la colonne : 'Timestamp'
#transactions_processing.loc[transactions_processing['Timestamp'] >= datetime.datetime(year=2022, month=3, day=27)]
#transactions_processing.loc[transactions_processing['Timestamp'] >= datetime(year=2021, month=10, day=31, hour=2)]
transactions_processing['Timestamp'].dtype
transactions_processing.head(1)
transactions_processing.set_index(pd.DatetimeTZDtype(transactions_processing['Timestamp'], year=2021, month=3, day=1 ,tz="Europe/Paris"), inplace=True)
transactions_processing
#x = datetime.datetime.strptime(transactions_processing['Timestamp'], "%Y-%m-%d %H:%M:%S.%f")

#target_timezone = pytz.timezone("Europe/Paris")

transactions_processing['Timestamp'] = transactions_processing['Timestamp'].tz_localize(pytz.timezone("Europe/Paris"), axis="index", ambiguous='infer') 
#localized_paris = target_timezone.localize(transactions_processing['Timestamp'])

#localized_paris

#transactions_processing['Timestamp'].apply(lambda x: )
#for ts in tz_conversion:
    #dt_paris = datetime.datetime.strptime(ts, "%Y-%m-%d %H:%M:%S.%f")
    #current_timezone = pytz.timezone("Europe/Paris")
    #target_timezone = pytz.timezone("Europe/Paris")
#localized_paris = current_timezone.localize(dt_paris)
    #localized_paris.astimezone(target_timezone)
    #print(tz_conversion)
    

date_time_lapage = transactions_processing['Timestamp']

#on défini une datetime en string
datetime_string = "2021-03-01 00:01:07.843138"

#Cette actien va prendre le datetime en string avec le format associé
datetime_paris = datetime.datetime.strptime(date_time_lapage, "%Y-%m-%d %H:%M:%S.%f")

#le timezone de départ
current_timezone = pytz.timezone("Europe/Paris")

#le timezonez voulu
target_timezone = pytz.timezone("Europe/Paris")

#je prend mon time zone de départ et je choisi à quelle valeur je l'applique avec un format explicit
localized_paris = current_timezone.localize(datetime_paris)

#je converti au timezone target (Paris)
localized_paris.astimezone(target_timezone)

transactions_processing
transactions_processing.set_index(pd.to_datetime(transactions_processing['Timestamp']), inplace=True)
#transactions_processing.drop(columns='Timestamp', inplace=True)
#transactions_processing
#transactions_processing.index.dtype

#france_tz = timezone('Europe/Paris')
#timestamp = pd.Timestamp('2023-09-28 24:21:23.82736', year=2023, month=9, day=28, hour=24, minute=21, second=23, microsecond=82736, fold=0, )

#timestamp = pd.Timestamp('2023-09-28 24:21:23.82736')
#shifted_timestamp = timestamp - pd.Timedelta(hours=1)
#print(shifted_timestamp) 
transactions_processing.loc[(transactions_processing['Timestamp'] > datetime(year=2022, month=10, day=30)) & (transactions_processing['Timestamp'] < datetime(year=2022, month=10, day=30, hour=3))]

transactions_processing[transactions_processing.duplicated(subset='Timestamp', keep=False)]
# première timestamp enregistrer le 1er mars 2021
# Je dois vérifier quelle heure est le temps standard en france
# D'après mes recherche, c'est l'heure d'hiver qui est la bonne (Dernier dimanche d'octobre -> dernier dimanche de mars)
# Comme le premier enregistrement du site à lieu le 1er mars 2021, c'est bien un horaire standard
transactions_processing.head(1)
#Je cherche l'index qui à été enregistrer en dernier le 28 mars 2021 à 01:59:59 max
transactions_processing[transactions_processing['date'].str.contains('2021-03-28 01')].tail(1)# et je regarde l'index du dernier enregistrement
#J'affiche également l'index qui suit pour vérifier qu'il y a bien eu changement d'heure
transactions_processing.iloc[[25063, 25065]]
#Il y a bien eu changement d'heure
#je dois donc réculer d'1h tout les enregistrement entre le 28 mars 2021 à 2h du matin et le 31 octobre 2021 à 3h du matin
#On voit qu'il y a deux fois plus d'enregistrement que les autres heures 
transactions_processing[transactions_processing['date'].str.contains('2021-10-31 01')].tail(1)# et je regarde l'index du dernier enregistrement
h_été = transactions_processing.iloc[25065:224942]
h_été.loc[h_été['Time'].str.startswith('24')]
#je localise les lignes d'heures d'été 

#transactions_processing[transactions_processing.iloc[25065:224942, 1]]
#timestamp = pd.Timestamp(heure_été['date'], )
#shifted_timestamp = timestamp - pd.Timedelta(hours=1)
#Les ligne qui dépassent l'heure réglementaire de 23:59:59
#transactions_processing[transactions_processing['date'].str.contains(' 24:')]
#Ca veut dire que ces heures devrait être reculer d'1heure
#Je dois trouver la date et l'heure à laquelle il y a eu un saut d'1 heure (entre 2 et 3h du matin)

